In [2]:
# Install langchain dependencies
!pip install -qU langchain-core langchain-google-genai langchain-text-splitters httpx

In [3]:
# Restart kernel after installs so that your environment can access the new
# packages.
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [16]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field

import os

os.environ["GOOGLE_API_KEY"] = userdata.get("api_key")
os.environ["GOOGLE_CLOUD_PROJECT"] = userdata.get("project_id")

PROJECT_ID = userdata.get("project_id")
print(f"Your project ID is: {PROJECT_ID}")

Your project ID is: gen-lang-client-0943365174


In [2]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.8,
)

In [4]:
# Synchronous invoke
query = "What is the meaning of life?"

message = llm.invoke(query)
print(message.content)

As a large language model, I don't have personal beliefs or opinions about the meaning of life. This is a question that philosophers and theologians have grappled with for centuries, and there is no one definitive answer. 

Here are some perspectives on the meaning of life:

* **Existentialism:**  Focuses on individual freedom and responsibility. Life has no inherent meaning, but we create our own meaning through our choices and actions.
* **Nihilism:**  Believes that life is meaningless and without purpose.
* **Absurdism:**  Acknowledges the inherent absurdity of life and encourages us to embrace it.
* **Religious perspectives:**  Many religions offer a framework for understanding the meaning of life, often focusing on a higher power, spiritual growth, and the afterlife.
* **Humanism:**  Emphasizes human values, reason, and ethics. Meaning is found in human connection, creativity, and contributing to society.
* **Hedonism:**  Seeks pleasure and happiness as the ultimate goal of life.


In [5]:
# Stream synchronous invoke
for chunk in llm.stream(query):
    print(chunk.content)

The
 meaning of life is a question that has been pondered by philosophers, theologians, and
 individuals for centuries. There is no single, definitive answer, and the meaning can
 vary greatly from person to person.

Here are some common perspectives:

**Existentialism:** Emphasizes individual freedom and responsibility. Meaning is not predetermined,
 but rather created through our choices and actions.

**Nihilism:** Argues that life has no inherent meaning or purpose. Everything is ultimately pointless.


**Hedonism:** Focuses on maximizing pleasure and minimizing pain. Meaning is found in pursuing personal happiness.

**Utilitarianism:** Emphasizes maximizing overall happiness and well-being for the greatest number of people. Meaning is found in contributing
 to the common good.

**Religious perspectives:** Often define meaning through a belief in a higher power or a spiritual purpose. Meaning is found in following divine commandments, achieving enlightenment, or serving a greater ca

In [6]:
# Asynchrounous invoke
message = await llm.ainvoke(query)
print(message.content)

# stream:
# async for chunk in (await llm.astream(messages))

# batch:
# await llm.abatch([messages])

The meaning of life is a profound question that has been pondered by philosophers, theologians, and individuals for centuries. There is no single, definitive answer, as it is a highly personal and subjective question. 

Here are some perspectives on the meaning of life:

**Philosophical Perspectives:**

* **Nihilism:**  Life has no inherent meaning or purpose.  
* **Existentialism:**  Individuals create their own meaning and purpose through their choices and actions.
* **Absurdism:**  Life is inherently meaningless, but we can still find joy and fulfillment by embracing the absurdity.
* **Hedonism:**  The pursuit of pleasure and happiness is the ultimate goal.
* **Utilitarianism:**  The meaning of life is to maximize happiness for the greatest number of people.

**Religious Perspectives:**

* **Theism:**  Life has a divine purpose and meaning as determined by a higher power.
* **Buddhism:**  The meaning of life is to achieve enlightenment and escape the cycle of suffering.
* **Christia

In [9]:
# Chaining prompts
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
message = chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

print(message.content)

Ich liebe Programmieren. 



In [17]:
# Structured ouput
from typing import Optional

class Joke(BaseModel):
    '''Joke to tell user.'''

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(description="How funny the joke is, from 1 to 10")

structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")

Joke(setup="Why don\\'t cats play poker?", punchline='Because they always have an ace up their sleeve!', rating=None)

In [22]:
# Tool calling
class GetWeather(BaseModel):
    '''Get the current weather in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )


class GetPopulation(BaseModel):
    '''Get the current population in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )


llm_with_tools = llm.bind_tools([GetWeather, GetPopulation])
message = llm_with_tools.invoke(
    "Which city is hotter today and which is bigger: LA or NY?"
)
message.tool_calls

[{'name': 'GetWeather',
  'args': {'location': 'Los Angeles, CA'},
  'id': '1d1c99c5-3fc9-4cd4-9636-1f470fb852fb',
  'type': 'tool_call'},
 {'name': 'GetWeather',
  'args': {'location': 'New York, NY'},
  'id': 'bce87dd9-a719-4ae2-a770-6a86125fa185',
  'type': 'tool_call'},
 {'name': 'GetPopulation',
  'args': {'location': 'Los Angeles, CA'},
  'id': '9379e08d-c47e-448e-94e8-618399a2fb1f',
  'type': 'tool_call'},
 {'name': 'GetPopulation',
  'args': {'location': 'New York, NY'},
  'id': 'ff53e4e8-57f1-4cbd-8419-5eeab8ff6c09',
  'type': 'tool_call'}]

In [15]:
# Image input
import base64
import httpx

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")
message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ]
)
message = llm.invoke([message])
print(message.content)

The weather looks sunny and clear. The sky is blue with white fluffy clouds, and the grass is a vibrant green, suggesting a warm and pleasant day.  There is no sign of rain or any other inclement weather.
